<a href="https://colab.research.google.com/github/mherskovitz/FRED/blob/main/fred_visualizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 FRED Corporate Bond Data Visualizer with Interactive UI

This notebook retrieves, plots, and downloads corporate yield and OAS data from FRED with an interactive user interface.

**Features:**
- Interactive date range selection
- Multiple series plotting with statistical overlays
- Dual-axis comparison charts
- Custom series support
- Automatic file downloads (PNG & HTML)

**Requirements:**
- FRED API key stored in Google Colab Secrets as 'FRED_API'

## 1. Install Required Libraries

In [10]:
!pip install fredapi plotly kaleido pandas_datareader ipywidgets


## 2. Import Libraries

In [2]:
import pandas as pd
import pandas_datareader.data as web
import plotly.express as px
import plotly.graph_objects as go
import datetime
from datetime import date, timedelta
from fredapi import Fred
from google.colab import userdata, files
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import os

## 3. FRED API Setup

In [3]:
# Access the FRED API key from Google Secrets
try:
    FRED_API_KEY = userdata.get('FRED_API')
    fred = Fred(api_key=FRED_API_KEY)
    print("✅ FRED API connected successfully!")
except Exception as e:
    raise Exception("❌ Failed to retrieve FRED API key from Google Secrets. Ensure the secret is set up correctly.") from e

✅ FRED API connected successfully!


## 4. Core Data Functions

In [4]:
def fetch_fred_data(series_id, start_date, end_date):
    """
    Fetch data and metadata from FRED using the series ID.

    Parameters:
    - series_id: str, the FRED series ID
    - start_date: str, start date in 'YYYY-MM-DD' format
    - end_date: str, end date in 'YYYY-MM-DD' format

    Returns:
    - pd.DataFrame: the fetched data
    - str: the series name (title)
    """
    try:
        # Fetch data using pandas_datareader
        data = web.DataReader(series_id, 'fred', start_date, end_date)

        # Fetch metadata using fredapi
        series_info = fred.get_series_info(series_id)
        series_name = series_info.get('title', series_id)

        return data, series_name
    except Exception as e:
        print(f"❌ Error fetching {series_id}: {str(e)}")
        raise

def create_statistical_plot(data, series_id, series_name):
    """Create a plotly figure with statistical overlays"""
    # Calculate statistics
    avg_rate = data[series_id].mean()
    std_dev = data[series_id].std()

    # Create base plot
    fig = px.line(data, x=data.index, y=series_id, title=series_name)

    # Add statistical lines
    fig.add_trace(go.Scatter(
        x=data.index, y=[avg_rate] * len(data),
        mode='lines', name='Average',
        line=dict(color='red', dash='dash')
    ))

    fig.add_trace(go.Scatter(
        x=data.index, y=[avg_rate + std_dev] * len(data),
        mode='lines', name='+1 Std Dev',
        line=dict(color='green', dash='dash')
    ))

    fig.add_trace(go.Scatter(
        x=data.index, y=[avg_rate - std_dev] * len(data),
        mode='lines', name='-1 Std Dev',
        line=dict(color='green', dash='dash')
    ))

    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Rate (%)',
        showlegend=True
    )

    return fig

## 5. Enhanced Plotting Functions

In [5]:
def plot_fred_data_enhanced(series_ids, start_date=None, end_date=None, output_dir='plots',
                           download_html=True, download_png=True):
    """Enhanced plotting function with flexible parameters"""

    # Set default dates if not provided
    if end_date is None:
        end_date = datetime.datetime.now()
    if start_date is None:
        start_date = end_date - datetime.timedelta(days=10 * 365)

    # Convert dates to strings if they're date objects
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d')
    else:
        start_date_str = start_date

    if hasattr(end_date, 'strftime'):
        end_date_str = end_date.strftime('%Y-%m-%d')
    else:
        end_date_str = end_date

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    for series_id in series_ids:
        try:
            # Fetch data
            data, series_name = fetch_fred_data(series_id, start_date_str, end_date_str)
            data = data.dropna()

            if data.empty:
                print(f"⚠️ No data available for {series_id} in the selected date range.")
                continue

            # Create plot
            fig = create_statistical_plot(data, series_id, series_name)

            # Save files
            if download_html:
                html_filepath = os.path.join(output_dir, f"{series_id}.html")
                fig.write_html(html_filepath)
                files.download(html_filepath)

            if download_png:
                png_filepath = os.path.join(output_dir, f"{series_id}.png")
                fig.write_image(png_filepath)
                files.download(png_filepath)

            # Display plot
            fig.show()
            print(f"✅ Successfully plotted {series_id}")

        except Exception as e:
            print(f"❌ Error plotting {series_id}: {str(e)}")

In [6]:
def plot_dual_axis_enhanced(yield_series_id, spread_series_id, start_date=None, end_date=None,
                           output_dir='plots', download_html=True, download_png=True):
    """Enhanced dual-axis plotting function"""

    # Set default dates if not provided
    if end_date is None:
        end_date = datetime.datetime.now()
    if start_date is None:
        start_date = end_date - datetime.timedelta(days=10 * 365)

    # Convert dates to strings if they're date objects
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d')
    else:
        start_date_str = start_date

    if hasattr(end_date, 'strftime'):
        end_date_str = end_date.strftime('%Y-%m-%d')
    else:
        end_date_str = end_date

    os.makedirs(output_dir, exist_ok=True)

    try:
        # Fetch both datasets
        yield_data, yield_name = fetch_fred_data(yield_series_id, start_date_str, end_date_str)
        spread_data, spread_name = fetch_fred_data(spread_series_id, start_date_str, end_date_str)

        # Combine data
        combined_data = pd.concat([
            yield_data[yield_series_id],
            spread_data[spread_series_id]
        ], axis=1).dropna()

        if combined_data.empty:
            print("⚠️ No overlapping data available for the selected series and date range.")
            return

        # Create dual-axis figure
        fig = go.Figure()

        # Add primary trace (left axis)
        fig.add_trace(go.Scatter(
            x=combined_data.index,
            y=combined_data[yield_series_id],
            name=yield_name,
            line=dict(color='blue')
        ))

        # Add secondary trace (right axis)
        fig.add_trace(go.Scatter(
            x=combined_data.index,
            y=combined_data[spread_series_id],
            name=spread_name,
            line=dict(color='red'),
            yaxis='y2'
        ))

        # Add average lines
        yield_avg = combined_data[yield_series_id].mean()
        spread_avg = combined_data[spread_series_id].mean()

        fig.add_trace(go.Scatter(
            x=combined_data.index,
            y=[yield_avg] * len(combined_data),
            name='Yield Average',
            line=dict(color='blue', dash='dash'),
            opacity=0.5
        ))

        fig.add_trace(go.Scatter(
            x=combined_data.index,
            y=[spread_avg] * len(combined_data),
            name='Spread Average',
            line=dict(color='red', dash='dash'),
            opacity=0.5,
            yaxis='y2'
        ))

        # Update layout
        fig.update_layout(
            title=f'Comparison: {yield_name} vs {spread_name}',
            xaxis_title='Date',
            yaxis=dict(
                title='Yield (%)',
                titlefont=dict(color='blue'),
                tickfont=dict(color='blue')
            ),
            yaxis2=dict(
                title='Spread (bps)',
                titlefont=dict(color='red'),
                tickfont=dict(color='red'),
                overlaying='y',
                side='right'
            ),
            showlegend=True
        )

        # Save files
        filename_base = f"comparison_{yield_series_id}_vs_{spread_series_id}"

        if download_html:
            html_filepath = os.path.join(output_dir, f"{filename_base}.html")
            fig.write_html(html_filepath)
            files.download(html_filepath)

        if download_png:
            png_filepath = os.path.join(output_dir, f"{filename_base}.png")
            fig.write_image(png_filepath)
            files.download(png_filepath)

        fig.show()
        print(f"✅ Successfully created comparison plot")

    except Exception as e:
        print(f"❌ Error creating comparison plot: {str(e)}")

## 6. Interactive User Interface

In [7]:
class FREDDataVisualizer:
    def __init__(self):
        self.series_options = {
            'BAMLH0A0HYM2': 'High Yield Corporate Bond Option-Adjusted Spread',
            'BAMLC0A0CMEY': 'Investment Grade Corporate Master Effective Yield',
            'BAMLC0A3CAEY': 'Investment Grade Corporate 3-5 Year Effective Yield',
            'BAMLC0A0CM': 'Investment Grade Corporate Option-Adjusted Spread',
            'DGS10': '10-Year Treasury Constant Maturity Rate',
            'DGS2': '2-Year Treasury Constant Maturity Rate',
            'BAMLC1A0C13Y': 'Investment Grade Corporate 1-3 Year Effective Yield',
            'BAMLC4A0C710Y': 'Investment Grade Corporate 7-10 Year Effective Yield'
        }
        self.setup_ui()

    def setup_ui(self):
        """Create the interactive user interface"""

        # Title
        self.title = widgets.HTML(
            value="<h2>📊 FRED Financial Data Visualizer</h2>",
            layout=widgets.Layout(margin='0 0 20px 0')
        )

        # Date controls
        self.start_date = widgets.DatePicker(
            description='Start Date:',
            value=date.today() - timedelta(days=10*365),
            style={'description_width': 'initial'}
        )

        self.end_date = widgets.DatePicker(
            description='End Date:',
            value=date.today(),
            style={'description_width': 'initial'}
        )

        # Series selection
        self.series_selector = widgets.SelectMultiple(
            options=[(f"{code}: {desc}", code) for code, desc in self.series_options.items()],
            value=['BAMLH0A0HYM2', 'BAMLC0A0CMEY'],
            description='Select Series:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(height='150px', width='600px')
        )

        # Custom series
        self.custom_series = widgets.Text(
            placeholder='Enter custom FRED series ID',
            description='Custom Series:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )

        # Comparison controls
        self.yield_series = widgets.Dropdown(
            options=[(f"{code}: {desc}", code) for code, desc in self.series_options.items()],
            value='BAMLC0A0CMEY',
            description='Yield Series:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )

        self.spread_series = widgets.Dropdown(
            options=[(f"{code}: {desc}", code) for code, desc in self.series_options.items()],
            value='BAMLH0A0HYM2',
            description='Spread Series:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )

        # Options
        self.output_dir = widgets.Text(
            value='fred_plots',
            description='Output Directory:',
            style={'description_width': 'initial'}
        )

        self.download_html = widgets.Checkbox(value=True, description='Download HTML')
        self.download_png = widgets.Checkbox(value=True, description='Download PNG')

        # Buttons
        self.plot_btn = widgets.Button(
            description='📈 Plot Selected Series',
            button_style='primary',
            layout=widgets.Layout(width='200px')
        )

        self.comparison_btn = widgets.Button(
            description='📊 Create Comparison',
            button_style='success',
            layout=widgets.Layout(width='200px')
        )

        self.add_custom_btn = widgets.Button(
            description='➕ Add Custom',
            button_style='info',
            layout=widgets.Layout(width='150px')
        )

        # Output
        self.output = widgets.Output()

        # Event handlers
        self.plot_btn.on_click(self.plot_series)
        self.comparison_btn.on_click(self.plot_comparison)
        self.add_custom_btn.on_click(self.add_custom)

        # Layout
        self.ui = widgets.VBox([
            self.title,
            widgets.HBox([self.start_date, self.end_date]),
            widgets.HTML("<br><b>📋 Series Selection:</b>"),
            self.series_selector,
            widgets.HBox([self.custom_series, self.add_custom_btn]),
            widgets.HTML("<br><b>🔄 Comparison Setup:</b>"),
            self.yield_series,
            self.spread_series,
            widgets.HTML("<br><b>⚙️ Options:</b>"),
            widgets.HBox([self.output_dir, self.download_html, self.download_png]),
            widgets.HTML("<br>"),
            widgets.HBox([self.plot_btn, self.comparison_btn]),
            self.output
        ])

    def add_custom(self, btn):
        """Add custom series"""
        custom_id = self.custom_series.value.strip().upper()
        if custom_id and custom_id not in self.series_options:
            try:
                series_info = fred.get_series_info(custom_id)
                series_name = series_info.get('title', custom_id)
                self.series_options[custom_id] = series_name

                # Update options
                new_options = [(f"{code}: {desc}", code) for code, desc in self.series_options.items()]
                self.series_selector.options = new_options
                self.yield_series.options = new_options
                self.spread_series.options = new_options

                self.custom_series.value = ''

                with self.output:
                    print(f"✅ Added: {custom_id} - {series_name}")

            except Exception as e:
                with self.output:
                    print(f"❌ Error adding {custom_id}: {str(e)}")

    def plot_series(self, btn):
        """Plot individual series"""
        with self.output:
            clear_output()
            selected = list(self.series_selector.value)
            if not selected:
                print("❌ Please select at least one series")
                return

            plot_fred_data_enhanced(
                series_ids=selected,
                start_date=self.start_date.value,
                end_date=self.end_date.value,
                output_dir=self.output_dir.value,
                download_html=self.download_html.value,
                download_png=self.download_png.value
            )

    def plot_comparison(self, btn):
        """Create comparison plot"""
        with self.output:
            clear_output()
            plot_dual_axis_enhanced(
                yield_series_id=self.yield_series.value,
                spread_series_id=self.spread_series.value,
                start_date=self.start_date.value,
                end_date=self.end_date.value,
                output_dir=self.output_dir.value,
                download_html=self.download_html.value,
                download_png=self.download_png.value
            )

    def display(self):
        """Display the UI"""
        display(self.ui)

## 7. Quick Start Functions (Backward Compatibility)

In [8]:
def quick_plot_series(series_ids=None):
    """Quick function to plot default series"""
    if series_ids is None:
        series_ids = ['BAMLH0A0HYM2', 'BAMLC0A0CMEY', 'BAMLC0A3CAEY']

    plot_fred_data_enhanced(series_ids)

def quick_comparison():
    """Quick function to create default comparison"""
    plot_dual_axis_enhanced('BAMLC0A0CMEY', 'BAMLH0A0HYM2')

print("🚀 FRED Data Visualizer Ready!")
print("\nQuick Start Options:")
print("1. Interactive UI: visualizer = FREDDataVisualizer(); visualizer.display()")
print("2. Quick plot: quick_plot_series()")
print("3. Quick comparison: quick_comparison()")

🚀 FRED Data Visualizer Ready!

Quick Start Options:
1. Interactive UI: visualizer = FREDDataVisualizer(); visualizer.display()
2. Quick plot: quick_plot_series()
3. Quick comparison: quick_comparison()


## 8. Launch Interactive UI

Run the cell below to start the interactive interface:

In [9]:
# Create and launch the interactive visualizer
visualizer = FREDDataVisualizer()
visualizer.display()

## 9. Alternative: Run Original Functions

If you prefer to use the original functions without the UI:

In [ ]:
# Example: Plot specific series with custom date range
# plot_fred_data_enhanced(
#     series_ids=['BAMLH0A0HYM2', 'BAMLC0A0CMEY'],
#     start_date=date(2020, 1, 1),
#     end_date=date.today(),
#     output_dir='my_plots'
# )

# Example: Create comparison chart
# plot_dual_axis_enhanced(
#     yield_series_id='BAMLC0A0CMEY',
#     spread_series_id='BAMLH0A0HYM2'
# )

print("💡 Uncomment and run the examples above to use the functions directly!")